<a href="https://colab.research.google.com/github/martadorozh/reddit/blob/main/reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

** Web scrapper for Reddit **

Introduction Task:

1.Write a Python script to scrape data from Reddit with the following parameters:
Title,
Content,
Number of comments.

2.Save the data in a NoSQL database like MongoDB.

### **Additional functionalities:**

- Configure the script to run data scraping once a day, with manual trigger option available.
- Implement basic filtering of posts based on a single criterion, such as the number of comments.

Install necessary packages first!
using pip install

In [ ]:
# Modules import
import praw
from pymongo import MongoClient
import schedule
from datetime import time, timedelta, datetime
import time as tm

In [ ]:
# Authenticate with Reddit API
reddit = praw.Reddit(
    #client_id="insert_client_id",
    #client_secret="insert_client_secret",
    #user_agent="my-app by u/insert_username"
)

In [ ]:
# Initialize MongoDB client
client = MongoClient("mongodb://localhost:27017/")
db = client['webscarpper']  # Database name
collection = db['reddit']     # Collection name

In [ ]:
subreddits = ['bigdata']  # choose subreddit

In [ ]:
# Fetch content from a subreddit
def scrape_and_store(min_comments=1):
    for subreddit_name in subreddits:
        subreddit = reddit.subreddit(subreddit_name)
        for submission in subreddit.new(limit=30):
            if submission.num_comments >= min_comments:
               post_data = {
                   "Title": submission.title,
                   "Content": submission.selftext,
                   "Comment count": submission.num_comments
               }
               collection.insert_one(post_data)

schedule.every().day.at("19:40").do(scrape_and_store)


# Manual trigger option
def manual_trigger():
    print("Manual data scraping triggered.")
    scrape_and_store()
    print("Manual data scraping completed.")

# Run manual trigger option
while True:
    print("\nPress 'm' to manually trigger data scraping or 'q' to quit.")
    choice = input("Enter your choice: ")
    if choice.lower() == 'm':
        manual_trigger()
    elif choice.lower() == 'q':
        break
    else:
        print("Invalid choice.")

while True:
    schedule.run_pending()
    tm.sleep(5)  # Check every minute